In [43]:
import pandas as pd
import plotly.express as px
from math import ceil
pd.set_option('display.width', 150)

In [44]:
df_uld = pd.read_csv('uld.csv')
df_uld['Volume'] = df_uld['Length'] * df_uld['Width'] * df_uld['Height']
print(df_uld)

  UID  Length  Width  Height  WeightLimit    Volume
0  U1     224    318     162         2500  11539584
1  U2     224    318     162         2500  11539584
2  U3     244    318     244         2800  18932448
3  U4     244    318     244         2800  18932448
4  U5     244    318     285         3500  22113720
5  U6     244    318     285         3500  22113720


In [45]:
total_uld_weight_limit = df_uld['WeightLimit'].sum()
total_uld_volume = df_uld['Volume'].sum()

print(f'Total ULD weight limit: {total_uld_weight_limit} kg')
print(f'Total ULD volume: {total_uld_volume} cm^3')

Total ULD weight limit: 17600 kg
Total ULD volume: 105171504 cm^3


In [46]:
df = pd.read_csv('packages.csv')
df['Volume'] = df['Length'] * df['Width'] * df['Height']
df['Density'] = df['Volume'] / df['Weight'] * 1e-3
df['Roundedness'] = df[['Length', 'Width', 'Height']].max(axis=1) / df[['Length', 'Width', 'Height']].min(axis=1)
print(df.head())

   PID  Length  Width  Height  Weight      Type  Cost  Volume    Density  Roundedness
0  P-1      99     53      55      61   Economy   176  288585   4.730902     1.867925
1  P-2      56     99      81      53  Priority    -1  449064   8.472906     1.767857
2  P-3      42    101      51      17  Priority    -1  216342  12.726000     2.404762
3  P-4     108     75      56      73   Economy   138  453600   6.213699     1.928571
4  P-5      88     58      64      93   Economy   139  326656   3.512430     1.517241


In [47]:
economy_count = (df['Type'] == 'Economy').sum()
priority_count = (df['Type'] == 'Priority').sum()
total_count = df.shape[0]

print(f'Total packages: {total_count}')
print(f'Economy packages: {economy_count} ({economy_count / total_count * 100:.2f}%)')
print(f'Priority packages: {priority_count} ({priority_count / total_count * 100:.2f}%)')

economy_df = df[df['Type'] == 'Economy']
priority_df = df[df['Type'] == 'Priority']

total_packages_weight = df['Weight'].sum()
economy_packages_weight = economy_df['Weight'].sum()
priority_packages_weight = priority_df['Weight'].sum()
print(f'Total packages weight: {total_packages_weight} kg ({total_packages_weight / total_uld_weight_limit:.1f}x ULD)')
print(f'Economy packages weight: {economy_packages_weight} kg ({economy_packages_weight / total_packages_weight * 100:.2f}%)')
print(f'Priority packages weight: {priority_packages_weight} kg ({priority_packages_weight / total_packages_weight * 100:.2f}%)')

total_packages_volume = df['Volume'].sum()
economy_packages_volume = economy_df['Volume'].sum()
priority_packages_volume = priority_df['Volume'].sum()
print(f'Total packages weight: {total_packages_volume} cm^3 ({total_packages_volume / total_uld_volume:.1f}x ULD)')
print(f'Economy packages weight: {economy_packages_volume} cm^3 ({economy_packages_volume / total_packages_volume * 100:.2f}%)')
print(f'Priority packages weight: {priority_packages_volume} cm^3 ({priority_packages_volume / total_packages_volume * 100:.2f}%)')

Total packages: 400
Economy packages: 297 (74.25%)
Priority packages: 103 (25.75%)
Total packages weight: 30662 kg (1.7x ULD)
Economy packages weight: 22948 kg (74.84%)
Priority packages weight: 7714 kg (25.16%)
Total packages weight: 171334228 cm^3 (1.6x ULD)
Economy packages weight: 127930866 cm^3 (74.67%)
Priority packages weight: 43403362 cm^3 (25.33%)


In [48]:
package_wt_avg, package_wt_std = df['Weight'].mean(), df['Weight'].std()
economy_package_wt_avg, economy_package_wt_std = economy_df['Weight'].mean(), economy_df['Weight'].std()
priority_package_wt_avg, priority_package_wt_std = priority_df['Weight'].mean(), priority_df['Weight'].std()
print(f'Average packages weight: {package_wt_avg:.2f} ± {package_wt_std:.2f} kg')
print(f'Average economy packages weight: {economy_package_wt_avg:.2f} ± {economy_package_wt_std:.2f} kg')
print(f'Average priority packages weight: {priority_package_wt_avg:.2f} ± {priority_package_wt_std:.2f} kg')

package_vol_avg, package_vol_std = df['Volume'].mean(), df['Volume'].std()
economy_package_vol_avg, economy_package_vol_std = economy_df['Volume'].mean(), economy_df['Volume'].std()
priority_package_vol_avg, priority_package_vol_std = priority_df['Volume'].mean(), priority_df['Volume'].std()
print(f'Average packages volume: {package_vol_avg:.2f} ± {package_vol_std:.2f} cm^3')
print(f'Average economy packages volume: {economy_package_vol_avg:.2f} ± {economy_package_vol_std:.2f} cm^3')
print(f'Average priority packages volume: {priority_package_vol_avg:.2f} ± {priority_package_vol_std:.2f} cm^3')

Average packages weight: 76.66 ± 47.50 kg
Average economy packages weight: 77.27 ± 47.42 kg
Average priority packages weight: 74.89 ± 47.92 kg
Average packages volume: 428335.57 ± 195992.36 cm^3
Average economy packages volume: 430743.66 ± 201509.28 cm^3
Average priority packages volume: 421391.86 ± 179889.05 cm^3


In [49]:
df_uld['AvgPkgCnt(wt)'] = (df_uld['WeightLimit'] / package_wt_avg).apply(ceil)
df_uld['AvgPkgCnt(vol)'] = (df_uld['Volume'] / package_vol_avg).apply(ceil)

wt_cum_sum = df['Weight'].sort_values().cumsum()
df_uld['MaxPkgCnt(wt)'] = df_uld['WeightLimit'].apply(lambda x: wt_cum_sum.searchsorted(x))

vol_cum_sum = df['Volume'].sort_values().cumsum()
df_uld['MaxPkgCnt(vol)'] = df_uld['Volume'].apply(lambda x: vol_cum_sum.searchsorted(x))

print(df_uld)

  UID  Length  Width  Height  WeightLimit    Volume  AvgPkgCnt(wt)  AvgPkgCnt(vol)  MaxPkgCnt(wt)  MaxPkgCnt(vol)
0  U1     224    318     162         2500  11539584             33              27             92              62
1  U2     224    318     162         2500  11539584             33              27             92              62
2  U3     244    318     244         2800  18932448             37              45            100              90
3  U4     244    318     244         2800  18932448             37              45            100              90
4  U5     244    318     285         3500  22113720             46              52            116             101
5  U6     244    318     285         3500  22113720             46              52            116             101


In [50]:
px.scatter(df, x='Weight', y='Volume', template='plotly_dark')

In [51]:
px.histogram(df, x='Volume', template='plotly_dark', nbins=15)

In [52]:
px.histogram(df, x='Weight', template='plotly_dark', nbins=25)

In [53]:
px.histogram(df, x='Density', template='plotly_dark', nbins=10)

In [54]:
px.histogram(df, x='Roundedness', template='plotly_dark', nbins=20)

In [55]:
px.histogram(economy_df, x='Cost', template='plotly_dark', nbins=25)

In [56]:
px.scatter(economy_df, x='Cost', y='Weight', template='plotly_dark')